<a id='top'></a>

# Watershed and Catchment elevation statistics and percent north facing landcover.

## Aspect and Percent North Covariates
Calculate aspect grid with the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid, or we can't calculate covariates at
 the watershed scale.
 * **~~aspect_rch = calculate mean aspect for stream reach (zonal statistics)~~**
 * **~~aspect_cat = calculate mean aspect over catchments (zonal statistics)~~**
 * **north_wtd = create 1/0 grid of north facing cells from aspect grid (north = aspects from 315-45 degrees), ~~~use
  weighted flow accumulation to sum north facing cells, divide by flow accumulation grid to get % of north facing
  cells in each watershed~~~ Use tabulate area to quantify area of North cells (VALUE_1) and non-North (VALUE_0) cells
  for each watershed and calculate percent north as north_wtd = row[VALUE_1]/(row[VALUE_1]+row[VALUE_0])*100 .**

## AKSSF Watershed and Catchment Elevation Metrics
Calculate elevation metrics for catchment/watershed with temperature data using zonal statistics as table.

### Catchment Elevation Metrics
* **cat_elev_mn = mean elevation for catchment**
* **cat_elev_min = minimum elevation for catchment**
* **cat_elev_max = max elevation for catchment**
* **cat_elev_std = standard deviation of elevation for catchment**

### Watershed Elevation Metrics
* **wtd_elev_mn = mean watershed elevation**
* **wtd_elev_min = min watershed elevation**
* **wtd_elev_max = max watershed elevation**
* **wtd_elev_sd (or cv) = standard deviation of watershed elevation**

## Import modules

In [1]:
import arcpy
import os
import datetime
import time
import sys
arcpy.env.overwriteOutput = True
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

18102021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\landcover
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Collect data and set working output locations
Set data_dir to folder containing all AKSSF regional subfolders/geoadatabases
* RS data folder: data_dir = r"W:\GIS\AKSSF"
* DM data folder: data_dir = r"D:\GIS_Temp\AKSSF_BeckyCopy\AKSSF"

In [2]:
# Set AKSSF Data directory
# data_dir = r"W:\GIS\AKSSF"

# dm local
data_dir = r"D:\GIS_Temp\AKSSF"

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces(workspace_type="Folder")
rois= []
print (regions)

# Path to create output folder/gdb
temppath = r"D:\GIS_temp" # Output folder
dirname = 'AKSSF_land_met'
tempgdbname = 'AKSSF_land_met.gdb'
temp_dir = os.path.join(temppath, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

['D:\\GIS_temp\\AKSSF\\Bristol_Bay', 'D:\\GIS_temp\\AKSSF\\Cook_Inlet', 'D:\\GIS_temp\\AKSSF\\Copper_River', 'D:\\GIS_temp\\AKSSF\\Kodiak', 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']
Working Folder already created... D:\GIS_temp\AKSSF_land_met
Output location already exists D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb


In [3]:
# Set regional workspaces from AKSSF data folder and store in list
arcpy.env.workspace = data_dir
regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

['D:\\GIS_temp\\AKSSF\\Bristol_Bay',
 'D:\\GIS_temp\\AKSSF\\Cook_Inlet',
 'D:\\GIS_temp\\AKSSF\\Copper_River',
 'D:\\GIS_temp\\AKSSF\\Kodiak',
 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']

## Create aspect and north grids if they do not already exists
### NHDPlus

In [29]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with NHDPlus elev data

# elrasters = []
# nhd_dat = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_NHDPlus\\extracts"
# for dirpath, dirname, filenames in arcpy.da.Walk(nhd_dat, 'RasterDataset'):
#     for filename in filenames:
#         if filename == "elev_cm.tif":
#             elras = os.path.join(dirpath, filename)
#             elrasters.append(elras)
#             print (f'Appending {elras} to list')
#
# spatial_ref = arcpy.Describe(elrasters[0]).spatialReference
# print (spatial_ref.name)
# # Merge all nhdplus elevation rasters together
# elevname = 'AKSSF_NHDPlus_elev_cm.tif'
# AKSSF_elev_cm = arcpy.MosaicToNewRaster_management(elrasters,temp_dir,elevname, spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_NHDPlus_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_cm, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_NHDPlus_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020202\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020301\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020302\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020401\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020402\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020501\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020502\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSS

### TauDEM - Create grids using Timms Composite DEM for entire TauDEM processed areas

In [31]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with TauDEM elev data
# compDEM = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\data\topography\AKSSF_Elevation_Composite_10m.tif"
# clip to study area
# AKSSF_elev_10m = r"D:\\GIS_temp\\AKSSF_Composite_10m_extract.tif"
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_Composite_10m_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_10m, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_Composite_10m_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))


## Begin GIS portion
- May be faster and save more space to create aspect rasters here using extract by mask?
- May be able to use aspect raster with tabulate intersection to calculate percent North (aspects from 315-45 degrees)

### <b>UPDATE - 2021-10-12 Do not run aspect metrics at this time. Only need to run percent North for watersheds</b>

In [5]:
from arcpy.sa import *
import re

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
cat_elev_ztables = []
wtd_elev_ztables = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
#xrois = ['Bristol_Bay']
for roi in rois:
# Loop through regional folders
    for region in regions:
        print (region)
        if roi in str(region):
            print(f'{region} found for {roi}')
            if roi in nhdplus_dat:
                # NHDPLus raster data for DM pc, variables will be overwritten below if they exist in source folder.
                asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_aspect.tif"
                elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_elev_cm.tif"
                nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif"
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID']
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID']

                print(f'NHD data: north raster - {nor_rast}')
                print(f'         aspect raster - {asp_rast}')
                print(f'      elevation raster - {elev_rast}')
                print('----------')

            elif roi in tauDem_dat:
                # TauDEM raster data for DM pc, variables will be overwritten below if they exist in source folder.
                nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_north_aspect.tif"
                asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_aspect.tif"
                elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_extract.tif"
                cat_cur_fields = ['cat_ID_txt', 'gridcode']
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID']
                print(f'TauDem data: north raster - {nor_rast}')
                print(f'         aspect raster - {asp_rast}')
                print(f'      elevation raster - {elev_rast}')
                print('----------')

            stats_fields = ['MEAN','MAX','MIN','STD']
            wtds = []
            cats = []
            # Start iter timing function
            iteration_start = time.time()
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    # Create list of watersheds
                    if (("wtd" in filename) and ("merge" not in filename)):
                        wtds.append(os.path.join(dirpath, filename))

                    # Set watershed merged watersheds dataset
                    elif 'wtds_merge'in filename:
                        wtd_merge = os.path.join(dirpath, filename)
                        wtdfieldnames = []
                        wtdlstFields = arcpy.ListFields(wtd_merge)
                        for field in wtdlstFields:
                            wtdfieldnames.append(field.name)
                        if 'cat_ID_txt' in wtdfieldnames:
                            print ('Id field already in dataset')
                            print('----------')
                        else:
                            print (f'Adding cat_ID_txt field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(wtd_merge,"cat_ID_txt",field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                                for row in cur:
                                    row[0] = str(row[1])
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

                    # Select elevation raster
                    elif 'elev.tif' == filename:
                        elev_rast = os.path.join(dirpath, filename)

                    # # Select aspect raster
                    # elif 'aspect' in filename:
                    #     asp_rast = os.path.join(dirpath, filename)

                    # Select north raster
                    elif 'north.tif' == filename:
                        nor_rast = os.path.join(dirpath, filename)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' in filename:
                        cats = os.path.join(dirpath,filename)
                        catlstfields = arcpy.ListFields(cats)
                        catfieldnames = []
                        catlstfields = arcpy.ListFields(cats)
                        for field in catlstfields:
                            catfieldnames.append(field.name)
                        if 'cat_ID_txt' in catfieldnames:
                            print ('Id field already in dataset')
                        else:
                            print (f'Adding cat_ID_txt field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(cats,"cat_ID_txt",field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                                for row in cur:
                                    row[0] = str(row[1])
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

            print (f'Calculating aspect metrics and percent north for catchments & watersheds of interest in {roi}'
                   f' region')
            print ('----------')
            print(f'Geodatabase: {gdb}')
            print ('----------')
            print (f'Elevation Raster: {elev_rast}')
            print ('----------')
            print (f'North Aspect Raster: {nor_rast}')
            print ('----------')
            print (f'{len(wtds)} Watersheds to process')
            print ('----------')
            print (f'Catchment intersect {cats} selected')
            print ('----------')

            try:
                # Set output names/paths for watershed and catchment tables
                wtd_merge_name = roi + "_Watersheds_Merge"
                wtd_merge_path = os.path.join(outgdb,wtd_merge_name)

                # # Aspect variables
                # wtd_merge_asp_table_name = roi + "_Watersheds_Merge_AspectZstats"
                # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
                # cat_asp_table_name = roi + "_Catchments_AspectZstats"
                # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

                # Percent North variables
                wtd_merge_pernorth_table_name = roi + "_Watersheds_Merge_PercentNorth"
                wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
                # cat_pernorth_table_name = roi + "_Catchments_PercentNorth"
                # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

                # Elevation variables
                wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
                wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
                cat_elev_table_name = roi + "_Catchments_ElevZstats"
                cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)

                if not arcpy.Exists(wtd_merge):
                    mergestart = time.time()
                    # Merge watersheds
                    wtd_merge = arcpy.Merge_management( wtds, wtd_merge_path ,add_source='ADD_SOURCE_INFO')
                    # Add wtd_id field
                    arcpy.AddField_management(wtd_merge,'cat_ID_txt',field_type='TEXT')
                    # Add region field
                    arcpy.AddField_management(wtd_merge,'region',field_type='TEXT')
                    # Populate watershed id and region information using update cursor - faster than field calc
                    with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','cat_ID_txt','region']) as cur:
                        for row in cur:
                            row[1] = re.findall('\d+', row[0])[0]
                            row[2] = roi
                            # Update
                            cur.updateRow(row)
                        del(row)
                    del(cur)
                    mergestop = time.time()
                    mergetime = int (mergestop - mergestart)
                    print(f'Watershed Merge for {roi} Elapsed time: ({datetime.timedelta(seconds=mergetime)})')
                    print('----------')
                else:
                    print (f'Merged watershed dataset {wtd_merge} already created')
                    print('----------')
                    
                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin zonal statistics min/mean/max std dev for watersheds and catchments in {roi} region')

                # Elevation Zonal statistics  for watersheds
                wtd_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = "cat_ID_txt",
                                                                in_value_raster = elev_rast,
                                                                out_table = wtd_merge_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
                # Update region field
                with arcpy.da.UpdateCursor(wtd_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed elev table to list
                wtd_elev_ztables.append(wtd_elev_metrics_table)

                # Elevation zonal statistics for catchments
                cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = "cat_ID_txt",
                                                                in_value_raster = elev_rast,
                                                                out_table = cat_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(cat_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment elev table to list
                cat_elev_ztables.append(cat_elev_metrics_table)

                # # Aspect Zonal statistics  for watersheds
                # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
                # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
                # wtd_asp_ztables.append(wtd_asp_metrics_table)

                # # Aspect Zonal statistics for catchments
                # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
                #                                                statistics_type='ALL')
                # cat_asp_ztables.append(cat_asp_metrics_table)

                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
                print('----------')

                # Tabulate Area with north grid and catchments/watersheds
                tabarea_start = time.time()
                print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
                # Percent North Tabulate area for watersheds
                wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field='cat_ID_txt',
                                                              in_class_data=nor_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_pernorth_table_path
                                                              )
                # Add region and percent north fields
                arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, 'wtd_north_per', field_type='Float')

                with arcpy.da.UpdateCursor(wtd_per_north_tabarea,['cat_ID_txt','region', 'VALUE_0',
                                                                        'VALUE_1', 'north_wtd']) as cur:
                    for row in cur:
                        row[1] = roi
                        row[4] = row[3]/(row[3]+row[2])*100
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed percent north table to list
                wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

                # # Percent North Tabulate Area for catchments
                # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_txt',
                #                                             in_class_data=nor_rast,"Value",
                #                                             out_table=cat_pernorth_table_path)

                # # Add and calculate region identifier field for catchment table
                # arcpy.AddField_management(cat_per_north_tabarea, 'cat_ID_txt', field_type='Float')
                # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, 'north_cat', field_type='Float')
                #
                # with arcpy.da.UpdateCursor(cat_per_north_tabarea,['cat_ID_txt','region', 'VALUE_0',
                #                                                         'VALUE_1', 'north_cat']) as cur:
                #     for row in cur:
                #         row[1] = roi
                #         row[4] = row[3]/(row[3]+row[2])*100
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # cat_pernorth_taba_tables.append(cat_per_north_tabarea)
                tabarea_stop = time.time()
                tabarea_time = int (tabarea_stop - tabarea_start)
                print(f'Tabulate area for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
                print('----------')

            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

    else:
        print(f'Region {str(roi)} not found in {region}')
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



D:\GIS_temp\AKSSF\Bristol_Bay
D:\GIS_temp\AKSSF\Cook_Inlet
D:\GIS_temp\AKSSF\Cook_Inlet found for Cook_Inlet
NHD data: north raster - D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif
         aspect raster - D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_aspect.tif
      elevation raster - D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_elev_cm.tif
----------
Adding cat_ID_txt field to catchment dataset D:\GIS_temp\AKSSF\Cook_Inlet\Cook_Inlet.gdb\cats_intersect
----------
Adding cat_ID_txt field to watershed dataset D:\GIS_temp\AKSSF\Cook_Inlet\Cook_Inlet.gdb\wtds_merge
----------
Calculating aspect metrics and percent north for catchments & watersheds of interest in Cook_Inlet region
----------
Geodatabase: ['D:\\GIS_temp\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb']
----------
Elevation Raster: D:\GIS_temp\AKSSF\Cook_Inlet\elev.tif
----------
North Aspect Raster: D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif
----------
241 Watersheds to process
----------
Catchment intersect D:\GIS_te

## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [50]:
wtd_pernorth_taba_tables
cat_elev_ztables
wtd_elev_ztables

# Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_cat_elev')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_wtd_elev')

# Merge tables together
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)

In [51]:
#Set up field dictionary
dict = { 'ZONE_CODE': ('cat_elev_ZONE_CODE', 'wtd_elev_ZONE_CODE'),
 'COUNT': ('cat_elev_COUNT', 'wtd_elev_COUNT'),
 'AREA': ('cat_elev_AREA', 'wtd_elev_AREA'),
 'MIN': ('cat_elev_MIN', 'wtd_elev_MIN'),
 'MAX': ('cat_elev_MAX', 'wtd_elev_MAX'),
 'RANGE': ('cat_elev_RANGE', 'wtd_elev_RANGE'),
 'MEAN': ('cat_elev_MEAN', 'wtd_elev_MEAN'),
 'STD': ('cat_elev_STD', 'wtd_elev_STD'),
 'SUM': ('cat_elev_SUM', 'wtd_elev_SUM'),
 'VARIETY': ('cat_elev_VARIETY', 'wtd_elev_VARIETY'),
 'MAJORITY': ('cat_elev_MAJORITY', 'wtd_elev_MAJORITY'),
 'MINORITY': ('cat_elev_MINORITY', 'wtd_elev_MINORITY'),
 'MEDIAN': ('cat_elev_MEDIAN', 'wtd_elev_MEDIAN'),
 'PCT90': ('cat_elev_PCT90', 'wtd_elev_PCT90')}

catz = []
catzup = []
wtdz = []
wtdzup = []

# Field lists
wtd_pernoth_fields = arcpy.ListFields(wtd_per_north)
cat_elev_zfields = arcpy.ListFields(cat_elev)
wtd_elev_zfields = arcpy.ListFields(wtd_elev)

# for field in cat_elev_zfields:
#     fnam = field.name
#     print (fnam)
#     catznew = 'cat_elev_' + str(fnam)
#     catz.append(fnam)
#     catzup.append(catznew)
#
# for field in wtd_elev_zfields:
#     fnam = field.name
#     print (fnam)
#     wtdznew = 'wtd_elev_' + str(fnam)
#     wtdz.append(fnam)
#     wtdzup.append(wtdznew)

In [62]:
# Rename fields
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in dict:
        newname = dict[keyval][1]
        newalias = dict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in dict:
        newname = dict[keyval][0]
        newalias = dict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

In [114]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
#create empty dataframes
cat_df = pd.DataFrame()
wtd_df = pd.DataFrame()
wtd_n_df = pd.DataFrame()
#create empty field lists
cat_field_list = []
wtd_field_list = []
wtd_n_field_list = []

# Make catchment elev df
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
    print (field.name, field.aliasName)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","cat_elev_ZONE_CODE"],axis=1)
cat_df = cat_df.set_index('cat_ID_txt')

# Make watershed elev df
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)
wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","wtd_elev_ZONE_CODE"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_txt')

# Make watershed north df
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_txt')


OBJECTID OBJECTID
cat_ID_txt cat_ID_txt
cat_elev_ZONE_CODE cat_elev_ZONE_CODE
cat_elev_COUNT cat_elev_COUNT
cat_elev_AREA cat_elev_AREA
cat_elev_MIN cat_elev_MIN
cat_elev_MAX cat_elev_MAX
cat_elev_RANGE cat_elev_RANGE
cat_elev_MEAN cat_elev_MEAN
cat_elev_STD cat_elev_STD
cat_elev_SUM cat_elev_SUM
cat_elev_VARIETY cat_elev_VARIETY
cat_elev_MAJORITY cat_elev_MAJORITY
cat_elev_MINORITY cat_elev_MINORITY
cat_elev_MEDIAN cat_elev_MEDIAN
cat_elev_PCT90 cat_elev_PCT90
region region


KeyError: "None of ['cat_ID_txt'] are in the columns"

In [115]:
cat_df.sort_values(by='region')

,cat_elev_COUNT,cat_elev_AREA,cat_elev_MIN,cat_elev_MAX,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,cat_elev_MINORITY,cat_elev_MEDIAN,cat_elev_PCT90,region
cat_ID_txt,,,,,,,,,,,,,,
88586,15636.00,1563600.00,5,30,25,15.32,7.09,239565.00,26,6,28,17,24,Bristol_Bay
34126,34571.00,3457100.00,15,66,51,21.51,9.36,743750.00,52,19,65,19,25,Bristol_Bay
885,4034.00,403400.00,16,49,33,29.53,8.05,119119.00,34,16,47,29,40,Bristol_Bay
20194,12085.00,1208500.00,212,479,267,256.92,46.73,3104820.00,267,215,437,238,317,Bristol_Bay
21055,10874.00,1087400.00,5,39,34,13.99,9.24,152167.00,35,8,17,10,31,Bristol_Bay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26464,10795.00,1079500.00,10,263,253,90.13,42.61,972986.00,244,108,222,93,136,Prince_William_Sound
18457,2358.00,235800.00,3,70,67,19.23,14.74,45342.00,68,8,70,13,42,Prince_William_Sound
44623,4300.00,430000.00,0,13,13,5.56,2.99,23918.00,14,6,13,6,9,Prince_William_Sound


In [116]:
wtd_df.sort_values(by='region')

,wtd_elev_COUNT,wtd_elev_AREA,wtd_elev_MIN,wtd_elev_MAX,wtd_elev_RANGE,wtd_elev_MEAN,wtd_elev_STD,wtd_elev_SUM,wtd_elev_VARIETY,wtd_elev_MAJORITY,wtd_elev_MINORITY,wtd_elev_MEDIAN,wtd_elev_PCT90,region
cat_ID_txt,,,,,,,,,,,,,,
3023044.0,1487091.00,148709100.00,13,819,806,272.79,142.84,405658810.00,806,226,16,242,481,Bristol_Bay
3033956.0,2731076.00,273107600.00,13,1455,1442,517.22,293.48,1412562475.00,1443,13,1455,477,937,Bristol_Bay
3034126.0,792503.00,79250300.00,15,805,790,108.76,139.12,86193506.00,791,21,770,38,316,Bristol_Bay
4000885.0,4034.00,403400.00,16,49,33,29.53,8.05,119119.00,34,16,47,29,40,Bristol_Bay
4020194.0,12085.00,1208500.00,212,479,267,256.92,46.73,3104820.00,267,215,437,238,317,Bristol_Bay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26464.0,50434.00,5043400.00,10,3324,3314,600.64,831.39,30292661.00,1766,108,865,353,2568,Prince_William_Sound
18457.0,131871.00,13187100.00,1,3324,3323,193.96,569.45,25578255.00,1077,1,2478,72,157,Prince_William_Sound
44623.0,354677.00,35467700.00,0,3324,3324,451.24,473.36,160046103.00,2099,19,1248,376,960,Prince_William_Sound


In [117]:
wtd_n_df.sort_values(by='region')

,CAT_ID_TXT,VALUE_0,VALUE_1,region,north_wtd
295,3023044.0,116279900.00,32429200.00,Bristol_Bay,21.81
299,3033956.0,189873600.00,83234000.00,Bristol_Bay,30.48
300,3034126.0,52052800.00,27197500.00,Bristol_Bay,34.32
301,4000885.0,357100.00,46300.00,Bristol_Bay,11.48
302,4020194.0,1151200.00,57300.00,Bristol_Bay,4.74
...,...,...,...,...,...
412,26464.0,3100800.00,1942600.00,Prince_William_Sound,38.52
411,18457.0,10389600.00,2797500.00,Prince_William_Sound,21.21
428,44623.0,22243000.00,13224700.00,Prince_William_Sound,37.29
419,38993.0,3407700.00,294600.00,Prince_William_Sound,7.96


In [111]:
landmet_merge_df = pd.merge(wtd_df,cat_df,on='cat_ID_txt', how='outer')

,wtd_elev_COUNT,wtd_elev_AREA,wtd_elev_MIN,wtd_elev_MAX,wtd_elev_RANGE,wtd_elev_MEAN,wtd_elev_STD,wtd_elev_SUM,wtd_elev_VARIETY,wtd_elev_MAJORITY,...,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,cat_elev_MINORITY,cat_elev_MEDIAN,cat_elev_PCT90,region_y
cat_ID_txt,,,,,,,,,,,,,,,,,,,,,
75004200007057.0,258728.00,6468200.00,0,144500,144500,43255.27,37427.70,11191350026.00,99715,500,...,90800,34649.43,25122.14,1333483286.00,27216,400,36,33377,70627,Cook_Inlet
75000200010547.0,5613443.00,140336075.00,13141,45160,32019,24260.98,6157.49,136187609252.00,31673,21432,...,586,13395.05,125.80,35255769.00,510,13294,13141,13388,13565,Cook_Inlet
75000200005606.0,333424.00,8335600.00,14141,20215,6074,16940.60,1153.71,5648402724.00,5996,17421,...,2003,15215.23,522.56,435277309.00,1903,15864,14141,15294,15853,Cook_Inlet
75000200002554.0,94100.00,2352500.00,14175,17227,3052,15846.43,575.76,1491148775.00,2825,16230,...,2641,15593.10,521.04,980322504.00,2566,15508,14178,15626,16209,Cook_Inlet
75000200017518.0,1790105.00,44752625.00,55716,197502,141786,119840.65,26050.70,214527347400.00,122471,128903,...,8103,58948.07,2148.32,279531767.00,3032,61427,55716,58486,62625,Cook_Inlet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75019800010313.0,22480838.00,562020950.00,72462,280991,208529,122419.95,37416.27,2752102974453.00,193923,88700,...,1257,73133.74,268.65,864733310.00,1165,73346,72462,73171,73466,Copper_River
75019800000406.0,294229.00,7355725.00,66017,72493,6476,69062.47,1480.28,20320182865.00,6474,70062,...,6476,69062.47,1480.28,20320182865.00,6474,70062,66220,69057,71091,Copper_River
75003900039073.0,230415.00,5760375.00,489,9943,9454,3060.51,1557.69,705188333.00,8339,2632,...,1902,1140.87,479.45,1955444.00,938,624,529,1092,1773,Copper_River


In [121]:
dfs = [wtd_df,cat_df]
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_txt'), dfs)

In [122]:
df_final

,wtd_elev_COUNT,wtd_elev_AREA,wtd_elev_MIN,wtd_elev_MAX,wtd_elev_RANGE,wtd_elev_MEAN,wtd_elev_STD,wtd_elev_SUM,wtd_elev_VARIETY,wtd_elev_MAJORITY,...,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,cat_elev_MINORITY,cat_elev_MEDIAN,cat_elev_PCT90,region_y
cat_ID_txt,,,,,,,,,,,,,,,,,,,,,
75004200007057.0,258728.00,6468200.00,0,144500,144500,43255.27,37427.70,11191350026.00,99715,500,...,90800,34649.43,25122.14,1333483286.00,27216,400,36,33377,70627,Cook_Inlet
75004300006312.0,6172261.00,154306525.00,5900,56900,51000,25127.05,11779.40,155090722985.00,50691,12600,...,3900,7654.30,935.80,440704003.00,3605,6900,5909,7400,9002,Cook_Inlet
75004300001906.0,3674807.00,91870175.00,300,77174,76874,40142.44,11984.06,147515731019.00,75573,37700,...,14321,2818.64,2135.86,34164787.00,4573,1000,301,2126,5218,Cook_Inlet
75004300000100.0,1158453.00,28961325.00,3305,41700,38395,15654.17,8590.35,18134625884.00,36389,9100,...,4495,5889.18,819.43,463731919.00,3963,6500,3305,5933,6800,Cook_Inlet
75004300004983.0,610455.00,15261375.00,2100,147384,145284,67339.95,33460.53,41108006739.00,126942,33600,...,45843,26556.24,10043.37,3015646404.00,35402,33600,2103,28233,38776,Cook_Inlet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75003900023674.0,155526.00,3888150.00,1357,67537,66180,16985.47,16387.75,2641682563.00,43720,4726,...,165,1585.92,40.31,269606.00,105,1583,1508,1583,1642,Copper_River
75003900062264.0,886085.00,22152125.00,2182,54687,52505,15531.18,13899.79,13761946382.00,51028,3513,...,1180,3139.22,110.48,21054744.00,614,3175,2182,3161,3244,Copper_River
75003900055316.0,293348.00,7333700.00,2943,57672,54729,21011.88,14481.52,6163794155.00,51198,3082,...,3971,3256.86,626.21,30822898.00,1344,3069,2943,3062,3621,Copper_River
